In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.window import Window
import time
from pyspark.sql import SparkSession

/tmp/ipykernel_84228/1510302545.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("test") \
       .config("spark.driver.memory", "80g")\
       .config("spark.executor.memory", "50g")\
       .config("spark.dirver.maxResultSize", "50g")\
       .config('spark.local.dir', '/dados10t/datalake/raw/checkpoint_dir')\
       .getOrCreate()
sc = spark.sparkContext
sc.setCheckpointDir('/dados10t/datalake/raw/checkpoint_dir')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/23 20:37:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/23 20:37:21 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/10/23 20:37:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# srag

In [4]:
#srag = spark.read.parquet("/dados10t/datalake/standard/omop/data_with_ids/dbsrag_tb_influd_long_09072022_v8-id_omop.parquet")
srag = spark.read.parquet('/dados10t/datalake/standard/omop/data_with_ids/dbsrag_tb_influd_long_09072022_v8-id_omop-v3.parquet/')

In [5]:
id_vigivac = spark.read.parquet("/dados10t/datalake/standard/vigivac/base_analise/id_paciente_to_id_vigivac/id_vigivac_id_paciente_09072022.parquet/")

In [6]:
srag = srag.join(id_vigivac, ['id_paciente'], 'inner')

In [7]:
test_vars = ['pcr_sars2', 'an_sars2', 'pcr_resul', 'res_an', 'pcr_vsr', 'pcr_rino', 'an_vsr', 'pos_pcrflu', 'pos_an_flu']

In [6]:
#
srag.select(test_vars).show()

+---------+--------+---------+------+-------+--------+------+----------+----------+
|pcr_sars2|an_sars2|pcr_resul|res_an|pcr_vsr|pcr_rino|an_vsr|pos_pcrflu|pos_an_flu|
+---------+--------+---------+------+-------+--------+------+----------+----------+
|     null|    null|        2|     5|   null|    null|  null|      null|      null|
|        1|    null|        1|     4|   null|    null|  null|         2|      null|
|        1|    null|        1|     5|   null|    null|  null|         2|      null|
|     null|    null|        2|     5|   null|    null|  null|      null|      null|
|     null|    null|        2|     2|   null|    null|  null|      null|      null|
|     null|    null|        2|     4|   null|    null|  null|      null|      null|
|     null|    null|        2|     5|   null|    null|  null|      null|      null|
|     null|    null|        9|     4|   null|    null|  null|      null|      null|
|        1|    null|        1|     5|   null|    null|  null|         9|    

In [12]:
#
srag.select(
    'pcr_sars2', 
    'an_sars2',
    'pcr_resul',
    'res_an',
    'pcr_vsr',
    'pcr_rino',
    'an_vsr',
    'pos_pcrflu',
    'pos_an_flu'
).filter(F.col('pcr_sars2').isNotNull()).show()

+---------+--------+---------+------+-------+--------+------+----------+----------+
|pcr_sars2|an_sars2|pcr_resul|res_an|pcr_vsr|pcr_rino|an_vsr|pos_pcrflu|pos_an_flu|
+---------+--------+---------+------+-------+--------+------+----------+----------+
|        1|    null|        1|  null|   null|    null|  null|         2|      null|
|        1|    null|        1|     5|   null|    null|  null|      null|      null|
|        1|    null|        1|     5|   null|    null|  null|         2|      null|
|        1|    null|        1|     5|   null|    null|  null|         2|      null|
|        1|    null|        1|     5|   null|    null|  null|      null|      null|
|        1|    null|        1|     4|   null|    null|  null|      null|      null|
|        1|    null|        1|     4|   null|    null|  null|         2|      null|
|        1|    null|        1|     4|   null|    null|  null|         2|      null|
|        1|    null|        1|     4|   null|    null|  null|      null|    

In [ ]:
# tipo teste
# not nul pcr_sars2 -> RT-PCR
# not nul an_sars2 -> Antigeno
# pcr_resul in 1, 2, 3, 5 -> RT-PCR
# res_an in 1, 2, 3, 5 -> Antigeno
# pcr_resul == 4|9 or res_an == 4|9 -> Sem teste

In [98]:
#
srag.printSchema()

root
 |-- st_notific: string (nullable = true)
 |-- sem_not: string (nullable = true)
 |-- sem_pri: string (nullable = true)
 |-- sg_uf_not: string (nullable = true)
 |-- id_regiona: string (nullable = true)
 |-- co_regiona: decimal(10,0) (nullable = true)
 |-- id_municip: string (nullable = true)
 |-- co_mun_not: string (nullable = true)
 |-- id_unidade: string (nullable = true)
 |-- co_uni_not: string (nullable = true)
 |-- cs_sexo: string (nullable = true)
 |-- nu_idade_n: decimal(10,0) (nullable = true)
 |-- tp_idade: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- cs_gestant: decimal(10,0) (nullable = true)
 |-- cs_raca: short (nullable = true)
 |-- cs_etinia: string (nullable = true)
 |-- cs_escol_n: decimal(10,0) (nullable = true)
 |-- sg_uf: string (nullable = true)
 |-- id_mn_resi: string (nullable = true)
 |-- co_mun_res: string (nullable = true)
 |-- cs_zona: short (nullable = true)
 |-- febre: short (nullable = true)
 |-- tosse: short (nullable = true

In [4]:
#
srag.count()

2487085

### define test type - measurement_concept_id 

In [8]:
srag = srag\
.withColumn('test_type',
            F.when( F.col('pcr_sars2').isNotNull(), 'rt_pcr')\
            .when(F.col('an_sars2').isNotNull(), 'antigeno')\
            .when(F.col('pcr_resul').isin([1, 2, 3, 5]),  'rt-pcr')\
            .when(F.col('res_an').isin([1, 2, 3, 5]), 'antigeno')\
            .when(F.col('pcr_resul').isin([4,9]) | F.col('res_an').isin([4,9]), None)
           )

In [9]:
srag = srag.filter(F.col('test_type').isNotNull())

In [ ]:
# resultado teste
# pcr_vsr=1|pcr_rino=1|an_vsr=1|pos_pcrflu=1|pos_an_flu=1 -> Negativo
# pcr_sars2=1|an_sars2=1 -> confirmado
# pcr_resul=1|pos_pcrflu in 2|9| is.na(pos_pcrflu) -> confirmado
# res_an=1| pos_an_flu in 2|9 is.na(pos_an_flu) -> confirmado
# pcr_resul=2|res_an=2 -> negativo

In [7]:
#
srag.count()

2259491

### define test result - value_as_concept_id

In [10]:
srag = srag\
.withColumn('test_result',
            F.when( F.col('pcr_vsr') == 1, 'negative')\
            .when( F.col('pcr_rino') == 1, 'negative')\
            .when( F.col('an_vsr') == 1, 'negative')\
            .when( F.col('pos_pcrflu') == 1, 'negative')\
            .when( F.col('pos_an_flu') == 1, 'negative')\
            .when( F.col('pcr_sars2') == 1, 'positive')\
            .when( F.col('an_sars2') == 1, 'positive')\
            .when( F.col('pcr_resul') == 1, 'positive')\
            .when( F.col('pos_pcrflu').isin([2, 9]), 'positive')\
            .when( F.col('res_an') == 1, 'positive')\
            .when( F.col('pos_an_flu').isin([2, 9]), 'positive')\
            .when(F.col('pcr_resul') == 2, 'negative')\
            .when(F.col('res_an') == 2, 'negative')
           )

In [11]:
srag = srag.filter(F.col('test_result').isNotNull())

In [10]:
#
srag.count()

2087916

In [10]:
#
srag.select('test_type', 'test_result').show()

+---------+-----------+
|test_type|test_result|
+---------+-----------+
|   rt_pcr|   positive|
|   rt-pcr|   negative|
|   rt_pcr|   positive|
|   rt-pcr|   negative|
|   rt_pcr|   positive|
|   rt-pcr|   negative|
|   rt-pcr|   negative|
|   rt-pcr|   negative|
|   rt_pcr|   positive|
|   rt_pcr|   positive|
|   rt-pcr|   negative|
|   rt-pcr|   negative|
|   rt_pcr|   positive|
|   rt_pcr|   positive|
|   rt_pcr|   positive|
| antigeno|   positive|
|   rt_pcr|   positive|
|   rt-pcr|   negative|
|   rt_pcr|   positive|
|   rt-pcr|   negative|
+---------+-----------+
only showing top 20 rows



In [61]:
#
srag.select(['test_type', 'test_result'] + test_vars).show(1000)

+---------+-----------+---------+--------+---------+------+-------+--------+------+----------+----------+
|test_type|test_result|pcr_sars2|an_sars2|pcr_resul|res_an|pcr_vsr|pcr_rino|an_vsr|pos_pcrflu|pos_an_flu|
+---------+-----------+---------+--------+---------+------+-------+--------+------+----------+----------+
|   rt_pcr|   positive|        1|    null|        1|     5|   null|    null|  null|      null|      null|
|     null|       null|     null|    null|     null|  null|   null|    null|  null|      null|      null|
|     null|       null|     null|    null|        4|     4|   null|    null|  null|      null|      null|
|   rt-pcr|   negative|     null|    null|        2|     4|   null|    null|  null|      null|      null|
|   rt_pcr|   positive|        1|    null|        1|     5|   null|    null|  null|         2|      null|
|   rt-pcr|   negative|     null|    null|        2|     5|   null|    null|  null|      null|      null|
|   rt_pcr|   positive|        1|    null|    

### measurement_date

In [12]:
srag = srag.withColumn('measurement_date', 
                      F.when(F.col('test_type') == 'rt_pcr', 
                            F.coalesce(
                                F.to_date(F.col('dt_pcr'), 'dd/MM/yyyy'), 
                                F.to_date(F.col('dt_coleta'), 'dd/MM/yyyy'), 
                                F.to_date(F.col('dt_notific'), 'dd/MM/yyyy')
                            )
                      )\
                       .when(F.col('test_type') == 'antigeno', 
                            F.coalesce(
                                F.to_date(F.col('dt_res_an'), 'dd/MM/yyyy'),
                                F.to_date(F.col('dt_coleta'), 'dd/MM/yyyy'),
                                F.to_date(F.col('dt_notific'), 'dd/MM/yyyy')
                            )
                            )
                      )

In [13]:
srag = srag.filter(F.col('measurement_date').isNotNull())

In [12]:
#
srag.count()

2087916

In [14]:
srag = srag.select('visit_occurrence_id', 'id_vigvac',  'test_type', 'test_result', 'measurement_date')

# sg

In [15]:
sg = spark.read.parquet("/dados10t/datalake/standard/omop/data_with_ids/sg_long_09072022_v9-id_omop.parquet/")

In [15]:
#
sg.count()

82896519

In [11]:
#
sg.printSchema()

root
 |-- dt_datanotificacao: date (nullable = true)
 |-- dt_datainiciosintomas: date (nullable = true)
 |-- dt_datateste: date (nullable = true)
 |-- condicoes: string (nullable = true)
 |-- sintomas: string (nullable = true)
 |-- classificacaofinal: string (nullable = true)
 |-- dt_dataencerramento: date (nullable = true)
 |-- evolucaocaso: string (nullable = true)
 |-- tipoteste: string (nullable = true)
 |-- resultadoteste: string (nullable = true)
 |-- codigotipoteste: string (nullable = true)
 |-- codigoresultadoteste: string (nullable = true)
 |-- estadonotificacao: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: double (nullable = true)
 |-- codigo_ibge_resid: string (nullable = true)
 |-- profissionalsaude: string (nullable = true)
 |-- cbo: string (nullable = true)
 |-- evolucaocaso_2: string (nullable = true)
 |-- classificacaofinal_2: string (nullable = true)
 |-- sexo_2: string (nullable = true)
 |-- racac

In [72]:
#
sg.select('dt_datateste', 'tipoteste', 'codigotipoteste', 'resultadoteste', 'codigoresultadoteste').show(10, truncate=False)

+------------+-----------------------+---------------+--------------+--------------------+
|dt_datateste|tipoteste              |codigotipoteste|resultadoteste|codigoresultadoteste|
+------------+-----------------------+---------------+--------------+--------------------+
|2021-05-31  |RT-PCR                 |1              |Detectável    |1                   |
|2021-03-24  |RT-PCR                 |1              |Detectável    |1                   |
|2022-01-13  |RT-PCR                 |1              |null          |null                |
|2022-01-24  |TESTE RÁPIDO - ANTÍGENO|3              |Reagente      |1                   |
|2021-01-26  |TESTE RÁPIDO - ANTÍGENO|3              |Detectável    |1                   |
|2022-01-19  |TESTE RÁPIDO - ANTÍGENO|3              |Reagente      |1                   |
|2021-02-16  |RT-PCR                 |1              |null          |null                |
|2021-02-16  |RT-PCR                 |1              |Detectável    |1                   |

### define test type - measurement_concept_id 

In [16]:
sg = sg.withColumn('test_type', 
              F.when (F.lower(F.col('tipoteste')).like('%antígeno%'), 'antigeno')\
              .when(F.lower(F.col('tipoteste')).like('%rt-pcr%'), 'rt-pcr')
             )
#.select('test_type', 'tipoteste').show(100, truncate=False)

In [17]:
sg = sg.filter(F.col('test_type').isNotNull())

In [19]:
#
sg.count()

56475307

### define test result - value_as_concept_id

In [18]:
sg = sg.withColumn('test_result', 
              F.when (F.lower(F.col('resultadoteste')).startswith('positivo'), 'positive')\
              .when(F.lower(F.col('resultadoteste')).startswith('detectável'), 'positive')\
              .when(F.lower(F.col('resultadoteste')).startswith('reagente'), 'positive')\
              .when(F.lower(F.col('resultadoteste')).like('%negativo%'), 'negative')\
              .when(F.lower(F.col('resultadoteste')).like('%não%'), 'negative')
             )

In [19]:
sg = sg.filter(F.col('test_result').isNotNull())

In [22]:
#
sg.count()

47977499

### measurement_date 

In [20]:
sg = sg.withColumn('measurement_date',
                   F.coalesce(
                       F.col('dt_datateste'),
                       F.col('dt_datanotificacao'),
                       F.col('dt_datainiciosintomas')
                   )
                  )

In [20]:
sg = sg.filter(F.col('measurement_date').isNotNull())

In [21]:
#
sg.count()

47977498

In [21]:
sg = sg.select('visit_occurrence_id', 'id_vigvac',  'test_type', 'test_result', 'measurement_date')

# union

In [22]:
sg_srag = sg.union(srag)

### measurement_type_concept_id 

In [23]:
# EHR
sg_srag = sg_srag.withColumn('measurement_type_concept_id', F.lit(32817))

### measurement_concept_id

In [24]:
# sg_srag = sg_srag.withColumn('measurement_concept_id', F.lit(3033385))
sg_srag = sg_srag.withColumn('measurement_concept_id', 
                             F.when(F.col('test_type') == 'rt-pcr', 3964944)\
                             .when(F.col('test_type') == 'antigeno', 757685))

In [25]:
sg_srag = sg_srag.filter(F.col('measurement_concept_id').isNotNull())

### value_as_concept_id

In [26]:
sg_srag = sg_srag.withColumn('value_as_concept_id', 
                             F.when(F.col('test_result') == 'positive', 45884084).otherwise(45878583)
                            )

In [29]:
sg_srag.count()

49285778

### person_id

In [27]:
person_id = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_person.parquet/")
person_id = person_id.withColumnRenamed('id_vigivac', 'id_vigvac')

In [28]:
srag_sg_id = sg_srag.join(person_id, 'id_vigvac', 'inner').select(sg_srag.columns + ['person_id'])

In [36]:
srag_sg_id.select('measurement_concept_id').show()

+----------------------+
|measurement_concept_id|
+----------------------+
|               3964944|
|                757685|
|                757685|
|               3964944|
|               3964944|
|                757685|
|                757685|
|               3964944|
|               3964944|
|               3964944|
|                757685|
|               3964944|
|                757685|
|                757685|
|               3964944|
|               3964944|
|               3964944|
|                757685|
|               3964944|
|               3964944|
+----------------------+
only showing top 20 rows



### measurement_id 

In [25]:
# full_id = srag_sg_id.withColumn("new_column", F.lit("ABC"))
# w = Window().partitionBy('new_column').orderBy(F.lit('A'))
# full_id_with_row = full_id.withColumn("measurement_id",F.row_number().over(w)).drop("new_column").checkpoint()

In [29]:
full_id_with_row = srag_sg_id.rdd.zipWithIndex().toDF().select(F.col("_1.*"),F.col("_2").alias('measurement_id'))

In [36]:
full_id_with_row.show()

+-------------------+------------+---------+-----------+----------------+---------------------------+----------------------+-------------------+--------------+
|visit_occurrence_id|   id_vigvac|test_type|test_result|measurement_date|measurement_type_concept_id|measurement_concept_id|value_as_concept_id|measurement_id|
+-------------------+------------+---------+-----------+----------------+---------------------------+----------------------+-------------------+--------------+
|            4974171|463856467979|   rt-pcr|   positive|      2021-05-31|                      32817|               3033385|           45884084|             0|
|            4974173|463856467983|   rt-pcr|   positive|      2021-03-24|                      32817|               3033385|           45884084|             1|
|            4974177|463856467990| antigeno|   positive|      2022-01-24|                      32817|               3033385|           45884084|             2|
|            4974179|463856467992| antig

In [43]:
full_id_with_row.printSchema()

root
 |-- visit_occurrence_id: long (nullable = true)
 |-- id_vigvac: long (nullable = true)
 |-- test_type: string (nullable = true)
 |-- test_result: string (nullable = true)
 |-- measurement_date: date (nullable = true)
 |-- measurement_type_concept_id: long (nullable = true)
 |-- measurement_concept_id: long (nullable = true)
 |-- value_as_concept_id: long (nullable = true)
 |-- person_id: long (nullable = true)
 |-- measurement_id: long (nullable = true)



In [30]:
full_id_with_row = full_id_with_row\
.withColumn('visit_occurrence_id', F.col('visit_occurrence_id').cast(IntegerType()))\
.withColumn('measurement_type_concept_id', F.col('measurement_type_concept_id').cast(IntegerType()))\
.withColumn('measurement_concept_id', F.col('measurement_concept_id').cast(IntegerType()))\
.withColumn('value_as_concept_id', F.col('value_as_concept_id').cast(IntegerType()))\
.withColumn('measurement_id', F.col('measurement_id').cast(IntegerType()))\
.withColumn('person_id', F.col('person_id').cast(IntegerType()))

In [ ]:
full_id_with_row.show()

# write

In [31]:
full_id_with_row\
.drop('id_vigvac')\
.drop('test_type')\
.drop('test_result')\
.write.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_measurement_data.parquet")

### condition

 |-- id_vigvac: long (nullable = true)
 |-- visit_occurrence_id: integer (nullable = true)
 |-- condition_start_date: date (nullable = true)
 |-- condition_concept_id: integer (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- condition_type_concept_id: integer (nullable = true)
 |-- condition_occurrence_id: integer (nullable = true)